In [2]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 88.9 MB/s eta 0:00:00


In [51]:
!python preprocess.py

✅ Preprocessing done. Uploaded train/test to S3.


In [9]:
import os

# Set required environment variables
os.environ['S3_BUCKET'] = 'fraud-detectml1'
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['FEATURE_GROUP_NAME'] = 'creditcard-fg1'
os.environ['MODEL_PACKAGE_GROUP_NAME'] = 'creditcard-mpg'
os.environ['ENDPOINT_NAME'] = 'creditcard-endpoint'

# Optional: Print to confirm
for var in ['S3_BUCKET', 'AWS_REGION', 'FEATURE_GROUP_NAME', 'MODEL_PACKAGE_GROUP_NAME', 'ENDPOINT_NAME']:
    print(f"{var} = {os.environ[var]}")


S3_BUCKET = fraud-detectml1
AWS_REGION = us-east-1
FEATURE_GROUP_NAME = creditcard-fg1
MODEL_PACKAGE_GROUP_NAME = creditcard-mpg
ENDPOINT_NAME = creditcard-endpoint


In [10]:
!python preprocess.py

Preprocessing done. Uploaded train/test to S3.


In [15]:
!python feature_store.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(
[04/08/25 22:55:32] INFO     Found credentials from IAM      ]8;id=104984;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=601428;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             Role:                                              
                             BaseNotebookInstanceEc2Instance                    
                             Role                                               
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
[04/08/2

In [18]:
!python train_xgboost.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(
[04/08/25 23:02:42] INFO     Found credentials from IAM      ]8;id=34908;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=495144;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             Role:                                              
                             BaseNotebookInstanceEc2Instance                    
                             Role                                               
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
[04/08/25

In [24]:
os.environ['TRAIN_JOB_NAME'] = 'trainingjobregister'


In [31]:
!python register_model.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(
[04/08/25 23:24:28] INFO     Found credentials from IAM      ]8;id=67765;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=432612;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             Role:                                              
                             BaseNotebookInstanceEc2Instance                    
                             Role                                               
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
[04/08/25

In [34]:
os.environ['SAGEMAKER_ROLE_ARN']="arn:aws:iam::419622399030:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"

In [40]:
!python deploy_endpoint.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(
[04/08/25 23:38:43] INFO     Found credentials from IAM      ]8;id=9089;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=900976;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             Role:                                              
                             BaseNotebookInstanceEc2Instance                    
                             Role                                               
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
          

In [45]:
import boto3

sm = boto3.client('sagemaker', region_name='us-east-1')  # Or your region
endpoints = sm.list_endpoints(SortBy='CreationTime', SortOrder='Descending')

for ep in endpoints['Endpoints']:
    print(f"➡️  {ep['EndpointName']} - Status: {ep['EndpointStatus']}")


➡️  creditcard-endpoint - Status: InService


In [46]:
import os
import boto3

runtime = boto3.client('sagemaker-runtime', region_name=os.environ['AWS_REGION'])
endpoint_name = os.environ['ENDPOINT_NAME']

# Example row of input features (must match model input)
# Adjust values and order to match your training dataset
sample_payload = "0.0,0.2,-1.3,1.4,0.5,0.0,0.0,0.2,-0.1,0.3,0.1,0.2,0.0,0.4,0.2,-0.5,0.6,-0.3,0.0,0.1,-0.4,0.2,0.1,0.0,150.0,0"  # Example

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=sample_payload
)

result = response['Body'].read().decode('utf-8')
print(f"🧠 Prediction: {result}")


🧠 Prediction: 0.9031805396080017



In [ ]:
      - python preprocess.py
      - python feature_store.py
      - python train_xgboost.py
      - python register_model.py
      - python deploy_endpoint.py

In [49]:
import os
import boto3
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# Load test data
test_df = pd.read_csv('s3://fraud-detectml1/data/test.csv')  # adjust path if needed
X_test = test_df.drop(columns=['Class'])  # assuming 'Class' is your label column
y_test = test_df['Class']

# Convert to CSV string (no header, no index)
payload = X_test.to_csv(index=False, header=False).strip()

# Endpoint name
endpoint_name = os.environ['ENDPOINT_NAME']
region = os.environ['AWS_REGION']

# Invoke endpoint
sm_runtime = boto3.client('sagemaker-runtime', region_name=region)
response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=payload
)

# Parse prediction response
result = response['Body'].read().decode('utf-8').strip().split('\n')
y_pred = np.array([int(float(pred)) for pred in result])

# Evaluate
acc = accuracy_score(y_test, y_pred)
print(f'✅ Endpoint Accuracy: {acc:.4f}')


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


✅ Endpoint Accuracy: 0.5025
